In [1]:
import h5py
import numpy as np
import pandas as pd
from astropy.time import Time
from tqdm import tqdm
import os
import vaex

In [2]:
def gps2dyr(time):
    """Converts GPS time to datetime (can also do decimal years)."""
    return Time(time, format='gps').datetime

In [3]:
def read_atl06(fname, cycle):
    """Read one ATL06 file and output 6 reduced files. 
    
    Extract variables of interest and separate the ATL06 file 
    into each beam (ground track) and ascending/descending orbits.
    """

    # Each beam is a group
    group = ['/gt1l', '/gt1r', '/gt2l', '/gt2r', '/gt3l', '/gt3r']
    
    # Loop trough beams
    dataframes = {}  # one dataframe per track
    
    with h5py.File(fname, 'r') as fi:
        # Check which ground tracks are present in this file
        gtracks = sorted(['/'+k for k in fi.keys() if k.startswith('gt')])
    
        for k, g in enumerate(gtracks): 
            # Read in data for a single beam
            data = {}
            # this is our unique key (per beam)
            data['id'] = fi[g+'/land_ice_segments/segment_id'][:]
            npts = len(data['id'])
            # Load vars into memory (include as many as you want)
            data['lat'] = fi[g+'/land_ice_segments/latitude'][:]
            data['lon'] = fi[g+'/land_ice_segments/longitude'][:]
            
            data['slope_y'] = fi[g+'/land_ice_segments/fit_statistics/dh_fit_dy'][:]
            data['slope_x'] = fi[g+'/land_ice_segments/fit_statistics/dh_fit_dx'][:]
            data['slope_x_sigma'] = fi[g+'/land_ice_segments/fit_statistics/dh_fit_dx_sigma'][:]

            data['h_li'] = fi[g+'/land_ice_segments/h_li'][:]
            data['s_li'] = fi[g+'/land_ice_segments/h_li_sigma'][:]
            data['q_flag'] = fi[g+'/land_ice_segments/atl06_quality_summary'][:]
            data['s_fg'] = fi[g+'/land_ice_segments/fit_statistics/signal_selection_source'][:]
            data['snr'] = fi[g+'/land_ice_segments/fit_statistics/snr_significance'][:]
            data['h_rb'] = fi[g+'/land_ice_segments/fit_statistics/h_robust_sprd'][:]
            data['bsnow_conf'] = fi[g+'/land_ice_segments/geophysical/bsnow_conf'][:]
            
            data['cloud_flg_asr'] = fi[g+'/land_ice_segments/geophysical/cloud_flg_asr'][:]
            data['cloud_flg_atm'] = fi[g+'/land_ice_segments/geophysical/cloud_flg_atm'][:]
            data['msw_flag'] = fi[g+'/land_ice_segments/geophysical/msw_flag'][:]
            data['fbsnow_h'] = fi[g+'/land_ice_segments/geophysical/bsnow_h'][:]
            data['bsnow_od'] = fi[g+'/land_ice_segments/geophysical/bsnow_od'][:]
            data['layer_flag'] = fi[g+'/land_ice_segments/geophysical/layer_flag'][:]
            data['bckgrd'] = fi[g+'/land_ice_segments/geophysical/bckgrd'][:]
            data['e_bckgrd'] = fi[g+'/land_ice_segments/geophysical/e_bckgrd'][:]
            data['n_fit_photons'] = fi[g+'/land_ice_segments/fit_statistics/n_fit_photons'][:]
            data['w_surface_window_final'] = fi[g+'/land_ice_segments/fit_statistics/w_surface_window_final'][:]
            
            delta_t = fi[g+'/land_ice_segments/delta_time'][:]     # for time conversion
            t_ref = fi['/ancillary_data/atlas_sdp_gps_epoch'][:]     # single value
            
            # Time in GPS seconds (secs since Jan 5, 1980)
            t_gps = t_ref + delta_t

            # GPS sec to datetime
            data['t_year'] = gps2dyr(t_gps)
            data['cycle'] = np.ones(npts, dtype=np.int8)*cycle
            data['track'] = np.repeat(g[1:], npts)
                
            # Make a dataframe out of our data dict and store it.
            dataframes[g] = pd.DataFrame.from_dict(data)
        
    return dataframes

In [4]:
pwd

'/home/jupyter/iceDivides'

In [5]:
seed = read_atl06('./ATL06_20190329071316_13870211_004_01.h5', cycle=int(3))

In [6]:
Pgt1l = seed.get('/gt1l').iloc[:2]
Pgt1r = seed.get('/gt1r').iloc[:2]
Pgt2l = seed.get('/gt2l').iloc[:2]
Pgt2r = seed.get('/gt2r').iloc[:2]
Pgt3l = seed.get('/gt3l').iloc[:2]
Pgt3r = seed.get('/gt3r').iloc[:2]

In [7]:
Pgt1l

,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,...,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track
0,1443587,-79.006042,176.344895,0.003497,0.045053,0.000830,-19.586994,0.012193,0,0,...,29.979246,0.268619,1,28285.419922,0.0,522,3.0,2019-03-29 07:13:52.971019,3,gt1l
1,1443588,-79.006216,176.344717,0.004264,0.050562,0.000883,-18.607821,0.012879,0,0,...,29.979246,0.268275,1,25933.798828,0.0,541,3.0,2019-03-29 07:13:52.973831,3,gt1l


In [8]:
gt1l = vaex.from_pandas(Pgt1l, copy_index=False)
gt1r = vaex.from_pandas(Pgt1r, copy_index=False)
gt2l = vaex.from_pandas(Pgt2l, copy_index=False)
gt2r = vaex.from_pandas(Pgt2r, copy_index=False)
gt3l = vaex.from_pandas(Pgt3l, copy_index=False)
gt3r = vaex.from_pandas(Pgt3r, copy_index=False)

In [11]:
files = os.listdir('./data03_12/')

In [12]:
for file in tqdm(files):
    dfs = read_atl06('./data03_12/' + file, cycle=int(3))
    if dfs.get('/gt1l', None) is not None:
        gt1l = gt1l.concat(vaex.from_pandas(dfs.get('/gt1l'), copy_index=False))
    if dfs.get('/gt1r', None) is not None:
        gt1r = gt1r.concat(vaex.from_pandas(dfs.get('/gt1r'), copy_index=False))
    if dfs.get('/gt2l', None) is not None:
        gt2l = gt2l.concat(vaex.from_pandas(dfs.get('/gt2l'), copy_index=False))
    if dfs.get('/gt2r', None) is not None:
        gt2r = gt2r.concat(vaex.from_pandas(dfs.get('/gt2r'), copy_index=False))
    if dfs.get('/gt3l', None) is not None:
        gt3l = gt3l.concat(vaex.from_pandas(dfs.get('/gt3l'), copy_index=False))
    if dfs.get('/gt3r', None) is not None:
        gt3r = gt3r.concat(vaex.from_pandas(dfs.get('/gt3r'), copy_index=False))

100%|██████████| 1320/1320 [46:52<00:00,  2.13s/it]


In [13]:
df = vaex.concat([gt1l, gt1r, gt2l, gt2r, gt3l, gt3r])

In [14]:
df

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track
0,1443587,-79.00604150693235,176.34489505559952,0.0034970236,0.045052722,0.0008298112,-19.586994,0.012193246,0,0,0.0,0.17003135,-1,2,1,3,29.979246,0.26861945,1,28285.42,0.0,522,3.0,2019-03-29 07:13:52.971019000,3,gt1l
1,1443588,-79.00621643747844,176.34471657932238,0.0042643934,0.050562415,0.00088303734,-18.60782,0.01287923,0,0,0.0,0.18802492,-1,2,1,3,29.979246,0.2682749,1,25933.799,0.0,541,3.0,2019-03-29 07:13:52.973831000,3,gt1l
2,1565320,-79.00590464294554,-29.15786824984113,-0.0038180242,0.008322206,0.0005828745,264.03226,0.011864527,0,0,0.0,0.131612,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,20022.52,0.0,390,3.0,2019-05-14 08:05:09.611856000,3,gt1l
3,1565321,-79.00572962014209,-29.158044121082156,-0.003440852,0.0070416494,0.000579469,264.19763,0.011928397,0,0,0.0,0.1270821,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,21030.754,0.0,376,3.0,2019-05-14 08:05:09.614699000,3,gt1l
4,1565322,-79.00555461855338,-29.158220563083404,-0.0032607878,0.008489967,0.0005951909,264.36688,0.01271345,0,0,0.0,0.13123506,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,19575.057,0.0,375,3.0,2019-05-14 08:05:09.617541000,3,gt1l
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"298,271,717",1569656,-78.2400490932243,-112.14232913945352,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.007984577,1,9233.532,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.767862000,3,gt3r
"298,271,718",1569657,-78.2398740994634,-112.14249557765775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.00819415,1,8526.938,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.770912000,3,gt3r
"298,271,719",1569752,-78.22323544581869,-112.157929146986,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,2,2,29.979246,-0.028147202,1,7081.393,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:35.038195000,3,gt3r
"298,271,720",1579859,-76.449328422305,-113.6012793545775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,1,1,3.4028235e+38,3.4028235e+38,1,7556.9717,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:04:03.526662000,3,gt3r


In [15]:
from numpy import r_
import pandas as pd

import mhealpy as mh

from numba import jit, int64, vectorize

In [16]:
def unique2parent(unique):
    '''
    Assumes input is UNIQ
    Currently only works on single resolution
    Returns parent base cell
    '''
    nside = mh.uniq2nside(unique)
    if nside.all():
        order = int(np.log2(nside.max()))
    unique = unique // 4**(order-1)
    parent = (unique - 16) // 4
    #print(order)
    return parent

def heal_norm(base, order, addr_nest):
    N_pix = mh.hp.order2nside(order)**2
    addr_norm = addr_nest - (base * N_pix)
    return addr_norm

@vectorize([int64(int64, int64)])#, target='cuda')
def VaexNorm2Mort(normed, parents):
    # Need to use vaex apply;
    # since we can't pass in 'order', we hard code it
    order = 18
    mask = np.int64(3*4**(order-1))
    res = np.zeros(order,dtype=np.int64)
    num = 0
    for j, i in enumerate(range(order, 0, -1)):
        nextBit = (normed & mask) >> ((2*i) - 2)
        num += (nextBit+1) * 10**(i-1)
        mask = mask >> 2
    parents = parents- 6
    parents = parents * 10**(order)
    num = num + parents
    return num

In [17]:
order = 18
uniq = mh.nest2uniq(2**order,mh.hp.ang2pix(2**order,df.lon.as_numpy().values, 
                                           df.lat.as_numpy().values,
                                           lonlat=True, nest=True))
parents = unique2parent(uniq)
normed = heal_norm(parents,order, uniq)

In [18]:
#uniq = mh.nest2uniq(2**18, nestcoords)
df['uniq'] = uniq
df['parents'] = parents
df['normed'] = normed

df['midx'] = df.apply(VaexNorm2Mort, arguments=[df.normed, df.parents], vectorize=True)

In [19]:
df

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track,uniq,parents,normed,midx
0,1443587,-79.00604150693235,176.34489505559952,0.0034970236,0.045052722,0.0008298112,-19.586994,0.012193246,0,0,0.0,0.17003135,-1,2,1,3,29.979246,0.26861945,1,28285.42,0.0,522,3.0,2019-03-29 07:13:52.971019000,3,gt1l,894772371480,9,276297080856,3112221322343111231
1,1443588,-79.00621643747844,176.34471657932238,0.0042643934,0.050562415,0.00088303734,-18.60782,0.01287923,0,0,0.0,0.18802492,-1,2,1,3,29.979246,0.2682749,1,25933.799,0.0,541,3.0,2019-03-29 07:13:52.973831000,3,gt1l,894772371469,9,276297080845,3112221322343111142
2,1565320,-79.00590464294554,-29.15786824984113,-0.0038180242,0.008322206,0.0005828745,264.03226,0.011864527,0,0,0.0,0.131612,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,20022.52,0.0,390,3.0,2019-05-14 08:05:09.611856000,3,gt1l,1032480798421,11,276566554325,5112321332334234222
3,1565321,-79.00572962014209,-29.158044121082156,-0.003440852,0.0070416494,0.000579469,264.19763,0.011928397,0,0,0.0,0.1270821,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,21030.754,0.0,376,3.0,2019-05-14 08:05:09.614699000,3,gt1l,1032480798421,11,276566554325,5112321332334234222
4,1565322,-79.00555461855338,-29.158220563083404,-0.0032607878,0.008489967,0.0005951909,264.36688,0.01271345,0,0,0.0,0.13123506,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,19575.057,0.0,375,3.0,2019-05-14 08:05:09.617541000,3,gt1l,1032480798594,11,276566554498,5112321332334243113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"298,271,717",1569656,-78.2400490932243,-112.14232913945352,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.007984577,1,9233.532,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.767862000,3,gt3r,963593850833,10,276399083473,4112233333421444212
"298,271,718",1569657,-78.2398740994634,-112.14249557765775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.00819415,1,8526.938,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.770912000,3,gt3r,963593850836,10,276399083476,4112233333421444221
"298,271,719",1569752,-78.22323544581869,-112.157929146986,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,2,2,29.979246,-0.028147202,1,7081.393,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:35.038195000,3,gt3r,963593861065,10,276399093705,4112233333424244132
"298,271,720",1579859,-76.449328422305,-113.6012793545775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,1,1,3.4028235e+38,3.4028235e+38,1,7556.9717,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:04:03.526662000,3,gt3r,963983610755,10,276788843395,4112412432332143114


In [20]:
midx = df.midx.as_numpy().values

In [21]:
del(df['midx'])
del(df['parents'])
del(df['normed'])

In [22]:
df['midx'] = midx
df

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track,uniq,midx
0,1443587,-79.00604150693235,176.34489505559952,0.0034970236,0.045052722,0.0008298112,-19.586994,0.012193246,0,0,0.0,0.17003135,-1,2,1,3,29.979246,0.26861945,1,28285.42,0.0,522,3.0,2019-03-29 07:13:52.971019000,3,gt1l,894772371480,3112221322343111231
1,1443588,-79.00621643747844,176.34471657932238,0.0042643934,0.050562415,0.00088303734,-18.60782,0.01287923,0,0,0.0,0.18802492,-1,2,1,3,29.979246,0.2682749,1,25933.799,0.0,541,3.0,2019-03-29 07:13:52.973831000,3,gt1l,894772371469,3112221322343111142
2,1565320,-79.00590464294554,-29.15786824984113,-0.0038180242,0.008322206,0.0005828745,264.03226,0.011864527,0,0,0.0,0.131612,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,20022.52,0.0,390,3.0,2019-05-14 08:05:09.611856000,3,gt1l,1032480798421,5112321332334234222
3,1565321,-79.00572962014209,-29.158044121082156,-0.003440852,0.0070416494,0.000579469,264.19763,0.011928397,0,0,0.0,0.1270821,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,21030.754,0.0,376,3.0,2019-05-14 08:05:09.614699000,3,gt1l,1032480798421,5112321332334234222
4,1565322,-79.00555461855338,-29.158220563083404,-0.0032607878,0.008489967,0.0005951909,264.36688,0.01271345,0,0,0.0,0.13123506,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,19575.057,0.0,375,3.0,2019-05-14 08:05:09.617541000,3,gt1l,1032480798594,5112321332334243113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"298,271,717",1569656,-78.2400490932243,-112.14232913945352,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.007984577,1,9233.532,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.767862000,3,gt3r,963593850833,4112233333421444212
"298,271,718",1569657,-78.2398740994634,-112.14249557765775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.00819415,1,8526.938,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.770912000,3,gt3r,963593850836,4112233333421444221
"298,271,719",1569752,-78.22323544581869,-112.157929146986,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,2,2,29.979246,-0.028147202,1,7081.393,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:35.038195000,3,gt3r,963593861065,4112233333424244132
"298,271,720",1579859,-76.449328422305,-113.6012793545775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,1,1,3.4028235e+38,3.4028235e+38,1,7556.9717,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:04:03.526662000,3,gt3r,963983610755,4112412432332143114


In [23]:
import pyproj

psgs = pyproj.Proj(3976)

In [24]:
lat = df.lat.to_numpy()
lon = df.lon.to_numpy()

In [25]:
x, y = psgs(lon,lat)

In [26]:
#df['x'], df['y'] = df.apply(psgs, arguments=[df.lon, df.lat], vectorize=True)

In [27]:
df['x'], df['y'] = x, y
df

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track,uniq,midx,x,y
0,1443587,-79.00604150693235,176.34489505559952,0.0034970236,0.045052722,0.0008298112,-19.586994,0.012193246,0,0,0.0,0.17003135,-1,2,1,3,29.979246,0.26861945,1,28285.42,0.0,522,3.0,2019-03-29 07:13:52.971019000,3,gt1l,894772371480,3112221322343111231,76147.62450344954,-1192036.1950727522
1,1443588,-79.00621643747844,176.34471657932238,0.0042643934,0.050562415,0.00088303734,-18.60782,0.01287923,0,0,0.0,0.18802492,-1,2,1,3,29.979246,0.2682749,1,25933.799,0.0,541,3.0,2019-03-29 07:13:52.973831000,3,gt1l,894772371469,3112221322343111142,76150.11884253839,-1192016.878556122
2,1565320,-79.00590464294554,-29.15786824984113,-0.0038180242,0.008322206,0.0005828745,264.03226,0.011864527,0,0,0.0,0.131612,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,20022.52,0.0,390,3.0,2019-05-14 08:05:09.611856000,3,gt1l,1032480798421,5112321332334234222,-581972.1311470547,1043116.9269472611
3,1565321,-79.00572962014209,-29.158044121082156,-0.003440852,0.0070416494,0.000579469,264.19763,0.011928397,0,0,0.0,0.1270821,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,21030.754,0.0,376,3.0,2019-05-14 08:05:09.614699000,3,gt1l,1032480798421,5112321332334234222,-581984.6527197912,1043131.844901119
4,1565322,-79.00555461855338,-29.158220563083404,-0.0032607878,0.008489967,0.0005951909,264.36688,0.01271345,0,0,0.0,0.13123506,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,19575.057,0.0,375,3.0,2019-05-14 08:05:09.617541000,3,gt1l,1032480798594,5112321332334243113,-581997.1836540232,1043146.7549702491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"298,271,717",1569656,-78.2400490932243,-112.14232913945352,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.007984577,1,9233.532,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.767862000,3,gt3r,963593850833,4112233333421444212,-1183965.1241931545,-481777.6041658824
"298,271,718",1569657,-78.2398740994634,-112.14249557765775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.00819415,1,8526.938,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.770912000,3,gt3r,963593850836,4112233333421444221,-1183981.4619760022,-481788.2611581817
"298,271,719",1569752,-78.22323544581869,-112.157929146986,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,2,2,29.979246,-0.028147202,1,7081.393,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:35.038195000,3,gt3r,963593861065,4112233333424244132,-1185537.9685311797,-482793.90209957986
"298,271,720",1579859,-76.449328422305,-113.6012793545775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,1,1,3.4028235e+38,3.4028235e+38,1,7556.9717,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:04:03.526662000,3,gt3r,963983610755,4112412432332143114,-1351174.6645843042,-590349.6253547024


In [28]:
df

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track,uniq,midx,x,y
0,1443587,-79.00604150693235,176.34489505559952,0.0034970236,0.045052722,0.0008298112,-19.586994,0.012193246,0,0,0.0,0.17003135,-1,2,1,3,29.979246,0.26861945,1,28285.42,0.0,522,3.0,2019-03-29 07:13:52.971019000,3,gt1l,894772371480,3112221322343111231,76147.62450344954,-1192036.1950727522
1,1443588,-79.00621643747844,176.34471657932238,0.0042643934,0.050562415,0.00088303734,-18.60782,0.01287923,0,0,0.0,0.18802492,-1,2,1,3,29.979246,0.2682749,1,25933.799,0.0,541,3.0,2019-03-29 07:13:52.973831000,3,gt1l,894772371469,3112221322343111142,76150.11884253839,-1192016.878556122
2,1565320,-79.00590464294554,-29.15786824984113,-0.0038180242,0.008322206,0.0005828745,264.03226,0.011864527,0,0,0.0,0.131612,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,20022.52,0.0,390,3.0,2019-05-14 08:05:09.611856000,3,gt1l,1032480798421,5112321332334234222,-581972.1311470547,1043116.9269472611
3,1565321,-79.00572962014209,-29.158044121082156,-0.003440852,0.0070416494,0.000579469,264.19763,0.011928397,0,0,0.0,0.1270821,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,21030.754,0.0,376,3.0,2019-05-14 08:05:09.614699000,3,gt1l,1032480798421,5112321332334234222,-581984.6527197912,1043131.844901119
4,1565322,-79.00555461855338,-29.158220563083404,-0.0032607878,0.008489967,0.0005951909,264.36688,0.01271345,0,0,0.0,0.13123506,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,19575.057,0.0,375,3.0,2019-05-14 08:05:09.617541000,3,gt1l,1032480798594,5112321332334243113,-581997.1836540232,1043146.7549702491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"298,271,717",1569656,-78.2400490932243,-112.14232913945352,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.007984577,1,9233.532,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.767862000,3,gt3r,963593850833,4112233333421444212,-1183965.1241931545,-481777.6041658824
"298,271,718",1569657,-78.2398740994634,-112.14249557765775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.00819415,1,8526.938,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.770912000,3,gt3r,963593850836,4112233333421444221,-1183981.4619760022,-481788.2611581817
"298,271,719",1569752,-78.22323544581869,-112.157929146986,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,2,2,29.979246,-0.028147202,1,7081.393,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:35.038195000,3,gt3r,963593861065,4112233333424244132,-1185537.9685311797,-482793.90209957986
"298,271,720",1579859,-76.449328422305,-113.6012793545775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,1,1,3.4028235e+38,3.4028235e+38,1,7556.9717,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:04:03.526662000,3,gt3r,963983610755,4112412432332143114,-1351174.6645843042,-590349.6253547024


In [29]:
df[2:]

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year,cycle,track,uniq,midx,x,y
0,1565320,-79.00590464294554,-29.15786824984113,-0.0038180242,0.008322206,0.0005828745,264.03226,0.011864527,0,0,0.0,0.131612,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,20022.52,0.0,390,3.0,2019-05-14 08:05:09.611856000,3,gt1l,1032480798421,5112321332334234222,-581972.1311470547,1043116.9269472611
1,1565321,-79.00572962014209,-29.158044121082156,-0.003440852,0.0070416494,0.000579469,264.19763,0.011928397,0,0,0.0,0.1270821,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,21030.754,0.0,376,3.0,2019-05-14 08:05:09.614699000,3,gt1l,1032480798421,5112321332334234222,-581984.6527197912,1043131.844901119
2,1565322,-79.00555461855338,-29.158220563083404,-0.0032607878,0.008489967,0.0005951909,264.36688,0.01271345,0,0,0.0,0.13123506,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,19575.057,0.0,375,3.0,2019-05-14 08:05:09.617541000,3,gt1l,1032480798594,5112321332334243113,-581997.1836540232,1043146.7549702491
3,1565323,-79.00537964808349,-29.158397841682415,-0.003500975,0.003751769,0.0006079925,264.49127,0.01261617,0,0,0.0,0.13588308,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,18078.805,0.0,368,3.0,2019-05-14 08:05:09.620379000,3,gt1l,1032480798594,5112321332334243113,-582009.7282627932,1043161.6535086596
4,1565324,-79.00520471356494,-29.158576080028098,-0.004122505,0.0056274175,0.0005678211,264.58536,0.012933277,0,0,0.0,0.12659779,-2,2,1,1,3.4028235e+38,3.4028235e+38,1,16378.219,0.0,379,3.0,2019-05-14 08:05:09.623214000,3,gt1l,1032480798601,5112321332334243132,-582022.2885315514,1043176.5388036236
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"298,271,715",1569656,-78.2400490932243,-112.14232913945352,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.007984577,1,9233.532,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.767862000,3,gt3r,963593850833,4112233333421444212,-1183965.1241931545,-481777.6041658824
"298,271,716",1569657,-78.2398740994634,-112.14249557765775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-3,5,2,2,29.979246,-0.00819415,1,8526.938,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:34.770912000,3,gt3r,963593850836,4112233333421444221,-1183981.4619760022,-481788.2611581817
"298,271,717",1569752,-78.22323544581869,-112.157929146986,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,2,2,29.979246,-0.028147202,1,7081.393,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:03:35.038195000,3,gt3r,963593861065,4112233333424244132,-1185537.9685311797,-482793.90209957986
"298,271,718",1579859,-76.449328422305,-113.6012793545775,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,3.4028235e+38,1,3,3.4028235e+38,3.4028235e+38,-1,5,1,1,3.4028235e+38,3.4028235e+38,1,7556.9717,3.4028235e+38,0,3.4028235e+38,2019-04-13 15:04:03.526662000,3,gt3r,963983610755,4112412432332143114,-1351174.6645843042,-590349.6253547024


In [30]:
df[2:].export_arrow('./atl06_03_12.arrow')

In [10]:
gt1l

#,id,lat,lon,slope_y,slope_x,slope_x_sigma,h_li,s_li,q_flag,s_fg,snr,h_rb,bsnow_conf,cloud_flg_asr,cloud_flg_atm,msw_flag,fbsnow_h,bsnow_od,layer_flag,bckgrd,e_bckgrd,n_fit_photons,w_surface_window_final,t_year
0,1443576,-79.00265175898602,152.6338938942904,-0.0034930306,-0.0019932939,0.0012799798,2112.0972,0.02197273,0,0,0.0,0.13674001,-1,1,0,0,98.97748,0.15880647,0,3543741.5,1567040.4,138,3.0,2018-12-28 13:08:17.510551000
1,1443577,-79.00282660535785,152.6337131144976,-0.0033072757,-0.005460351,0.0012954295,2111.9941,0.015203893,0,0,0.0,0.12070798,-1,1,0,0,96.86956,0.15470755,0,3428445.2,1567056.9,144,3.0,2018-12-28 13:08:17.513373000
2,1443581,-79.00770226316052,-70.68951907383446,0.014864666,0.008418874,0.0005335565,184.627,0.010601515,0,0,0.0,0.12233275,4,2,1,4,202.5068,0.23907205,0,821368.3,456252.12,397,3.0,2019-02-23 01:20:25.102900000
3,1443582,-79.00787704800452,-70.68970163845202,0.014720231,0.0057043494,0.0005324582,184.77682,0.011338408,0,0,0.0,0.12369104,4,2,1,4,200.39746,0.23566185,0,794943.7,456265.75,381,3.0,2019-02-23 01:20:25.105714000
4,1443583,-79.00805186741138,-70.68988326285287,0.014575872,0.009426143,0.0006639459,184.91608,0.012034703,0,0,0.0,0.14692499,4,2,1,4,198.28993,0.23225461,0,780100.6,456282.53,388,3.0,2019-02-23 01:20:25.108526000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"133,682,447",1565346,-79.00721045006628,-132.21977068251854,-0.006477784,0.004741233,0.0016565407,1102.3906,0.03348249,0,0,0.0,0.21513973,-1,4,1,2,209.85472,0.6916528,1,494481.06,420435.2,111,3.0,2019-03-28 17:10:59.581486000
"133,682,448",1565347,-79.00703565872891,-132.2199529490756,-0.005692235,0.002207704,0.0018826145,1102.4156,0.029799085,0,0,0.0,0.21348575,-1,4,1,2,209.85472,0.6932627,1,514356.84,420444.66,102,3.0,2019-03-28 17:10:59.584297000
"133,682,449",1565348,-79.00686084266192,-132.22013452710036,-0.0059905816,-0.003254145,0.0019431423,1102.4171,0.04439576,0,0,0.0,0.21254022,-1,4,1,2,209.85472,0.6948735,1,513579.88,420454.1,90,3.0,2019-03-28 17:10:59.587110000
"133,682,450",1565349,-79.00668598596165,-132.22031501663284,-0.006612603,0.0012488412,0.0021047168,1102.4423,0.037854582,0,0,0.0,0.21960956,-1,4,1,2,209.85472,0.6964865,1,496849.22,420463.56,92,3.0,2019-03-28 17:10:59.589927000
